## 모델, 데이터셋 준비

In [1]:
import sys
sys.path.append('/workspace')

In [2]:
from app.models.model_loader import load_asr_model

MODEL_VERSION = "v2"
IPA_ASR_MODEL = load_asr_model(MODEL_VERSION)

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import load_dataset, Audio

data_files = {
    'test': '/workspace/dataset/Test/data_for_test_feedback_algo.csv'
}

#! Custom Dataset 불러오기.
dataset = load_dataset("csv", data_files=data_files)

#! 오디오 파일을 audio 형식으로 처리
dataset = dataset.cast_column("audio", Audio())

print(dataset['test'])

Generating test split: 10 examples [00:00, 408.09 examples/s]

Dataset({
    features: ['audio', 'label'],
    num_rows: 10
})


## give_feedback API (개발 중)

In [4]:
from concurrent.futures import ThreadPoolExecutor
from app.hangul2ipa.worker import hangul2ipa
from app.feedback import get_asr_inference


"""
TODO 이 함수를 구현해야 함.
"""
def remove_noise_GPT(user_ipa, standard_ipa):
    user_ipa_clean = None
    return user_ipa_clean

"""
give_feedback (DEV) 함수가 하는 역할
1. 음성과 텍스트를 모두 IPA로 변환.
2. GPT를 통해 노이즈 제거.
3. 결과 반환.
"""

#! 원래 MODEL은 전역 변수, 여기서는 매개변수로 입력받음.
def give_feedback(audio, text, IPA_ASR_MODEL):
    
    #* Convert Audio and Text to IPA in parallel
    with ThreadPoolExecutor() as executor:
        user_ipa = executor.submit(get_asr_inference, IPA_ASR_MODEL, audio)
        standard_ipa = executor.submit(hangul2ipa, text)                                                         
    user_ipa = user_ipa.result()
    standard_ipa = standard_ipa.result()
    
    #! GPT를 통해 IPA에서 노이즈를 제거.
    user_ipa_clean = remove_noise_GPT(user_ipa, standard_ipa)
    
    feedback_data = {
        "user_ipa": user_ipa,
        "standard_ipa": standard_ipa,
        "user_ipa_clean" : user_ipa_clean,
    }

    return feedback_data

## give_feedback 테스트

In [6]:
for i, sample in enumerate(dataset['test']):
    audio = sample['audio']['path']
    label = sample['label']
    
    result = give_feedback(audio, label, IPA_ASR_MODEL)
    
    print(f"<{i+1}번쨰 샘플>")
    print(f"문장 : {label}")
    print(f"정답 IPA : {result['standard_ipa']}")
    print(f"사용자 IPA : {result['user_ipa']}")
    print()

<1번쨰 샘플>
문장 : 나는 행복하게 끝나는 영화가 좋다
정답 IPA : nɑnɯn hɛŋbokʰɑɡɛ k*ɯnnɑnɯn jʌŋwaɡɑ tɕotʰɑ
사용자 IPA : nɑnɯn hɛŋ[UNK]ʌkʰɑ[UNK]ɛ kɯnnɑnɯnjʌŋwa[UNK]ɑtɕotʰɑ

<2번쨰 샘플>
문장 : 휴가때 이탈리아에 여행 가고 싶어요
정답 IPA : hjuɡɑt*ɛ itʰɑlliɑɛ jʌɛŋ kɑɡo sipʰʌjo
사용자 IPA : sio[UNK]ɑpt*ɛɛ itʰɑ[UNK]i ɛijoɛŋkɑ[UNK]o sip pʌl

<3번쨰 샘플>
문장 : 세계 여행 하고싶어요
정답 IPA : sɛɡjɛ jʌɛŋ hɑɡosipʰʌjo
사용자 IPA : sɛ[UNK]ɛjʌŋɑ[UNK]o sippʰʌl

<4번쨰 샘플>
문장 : 나는 외출하기 전까지 방청소를 끝내야 한다
정답 IPA : nɑnɯn wɛtɕʰuɾɑɡi tɕʌŋk*ɑdʑi pɑŋtɕʰʌŋsoɾɯl k*ɯnnɛja hɑndɑ
사용자 IPA : mɑmɑnɰitɕʰu[UNK]ɑ[UNK]itɕʌŋ[UNK]ɑ[UNK][UNK]i pɑntɕʰʌŋsʌ[UNK]ɯl imnɛjahɯ[UNK]ɑ

<5번쨰 샘플>
문장 : 저는 러시아 음식을 요리할 줄 알아요
정답 IPA : tɕʌnɯn lʌsiɑ ɯmsiɡɯl joɾiɑl tɕul ɑɾɑjo
사용자 IPA : tɕʌnɑn ɯ[UNK]ʌ sɑn sɛ[UNK]ɯlɯ[UNK]jaɑn[UNK][UNK]ʌl ɑ[UNK]ɑja

<6번쨰 샘플>
문장 : 휴가 때 유럽에 여행가고 싶어요
정답 IPA : hjuɡɑ t*ɛ juɾʌbɛ jʌɛŋɡɑɡo sipʰʌjo
사용자 IPA : tʰju[UNK]ɑ[UNK]ɛju[UNK]ʌ[UNK]ɛjʌɛŋ kɑ[UNK]o sippʰʌʌ

<7번쨰 샘플>
문장 : 그 사람은 운동을 할 것 같지 않아
정답 IPA : kɯ sɑɾɑmɯn undoŋɯl hɑl kʌt kɑttɕ*i ɑnɑ
사용자 IPA : kɯ sɑ[UNK]ɑmɑn um[UNK][UNK]ʌŋɯl hɑl kot 